<a href="https://colab.research.google.com/github/LangqingZou/Math-509/blob/master/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [63]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rcParams
import seaborn as sns
import pylab as pl
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error,mean_squared_error, r2_score
from sklearn.model_selection import cross_val_score

# Data pre-process

In [38]:
columns = ['GFOFAIL','WEALTH','RESILIENCE','BELONG','HISCED','ST004D01T','IC001Q04TA','GCSELFEFF','ESCS','EMOSUPS','ST216Q03HA']
data = pd.read_csv('drive/MyDrive/Math_509_Project/CY07_MSU_STU_QQQ.csv',usecols=columns) 
data.head()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (17,229,323,871) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,ST004D01T,ST216Q03HA,IC001Q04TA,HISCED,ESCS,WEALTH,EMOSUPS,GFOFAIL,RESILIENCE,GCSELFEFF,BELONG
0,2,1,1,6,.6747,-0.0996,0.2127,-0.6870,0.3693,-0.4609,-0.2133
1,2,2,2,2,-.7566,-0.7221,-2.0851,-0.6133,-0.4684,0.7623,-0.0845
2,1,5,3,2,-2.5112,-3.6051,0.6364,-0.6133,-1.5248,-0.9477,-0.0824
3,2,2,1,2,-3.1843,-7.0376,1.0346,-0.6870,-0.0614,0.3313,-0.0358
4,2,2,1,4,-1.7557,-1.8375,-0.6576,0.1097,2.3693,0.6965,0.1704


In [4]:
data.dtypes

ST004D01T      object
ST216Q03HA     object
IC001Q04TA     object
HISCED          int64
ESCS           object
WEALTH        float64
EMOSUPS       float64
GFOFAIL       float64
RESILIENCE    float64
GCSELFEFF     float64
BELONG        float64
dtype: object

In [39]:
df = pd.DataFrame(data)
df = df.replace(r'^\s*$', np.nan, regex=True)
df = df.dropna() # drop all nan

# df_new = df.copy()
# df_new['HISCED'] = df_new['HISCED'].astype(object)

# change all object type to numeric type to calculate the correlation
df['ESCS'] = pd.to_numeric(df['ESCS'])
df['ST004D01T'] = pd.to_numeric(df['ST004D01T'])
df['ST216Q03HA'] = pd.to_numeric(df['ST216Q03HA'])
df['IC001Q04TA'] = pd.to_numeric(df['IC001Q04TA'])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [40]:
# dealing with the invalid values of 'HISCED'
# in 'HISCED' any value larger than 3 is invalid
df = df.loc[df['IC001Q04TA']<4]

In [41]:
# delete any rows contains value larger than 90
# since any value larger than 90 is invalid
mask = (df < 90)
df = df[mask].dropna()

In [ ]:
## save the dataset we need to a csv file
# df.to_csv("data_subset.csv")

In [ ]:
## we save the dataset with 12 variables into a subset
# data1 = pd.read_csv("data_subset.csv")
# data1

,Unnamed: 0,ST004D01T,ST216Q03HA,IC001Q04TA,HISCED,ESCS,WEALTH,EMOSUPS,GFOFAIL,RESILIENCE,GCSELFEFF,BELONG
0,0,2,1,1,6,0.6747,-0.0996,0.2127,-0.6870,0.3693,-0.4609,-0.2133
1,1,2,2,2,2,-0.7566,-0.7221,-2.0851,-0.6133,-0.4684,0.7623,-0.0845
2,2,1,5,3,2,-2.5112,-3.6051,0.6364,-0.6133,-1.5248,-0.9477,-0.0824
3,3,2,2,1,2,-3.1843,-7.0376,1.0346,-0.6870,-0.0614,0.3313,-0.0358
4,4,2,2,1,4,-1.7557,-1.8375,-0.6576,0.1097,2.3693,0.6965,0.1704
...,...,...,...,...,...,...,...,...,...,...,...,...
327751,606583,1,97,1,6,0.2900,0.7740,-0.6576,0.4637,-0.0614,99.0000,-0.3184
327752,606584,1,97,1,5,-0.3287,-0.1200,-2.0819,1.8905,-1.5350,99.0000,-1.8611
327753,606585,2,97,1,4,-2.0076,-1.3099,1.0346,0.4637,-0.5536,99.0000,-1.4766
327754,606586,1,97,1,4,-0.0691,-0.3349,1.0346,1.1422,-0.1433,99.0000,-1.0491


In [42]:
# check if deleting all nans
df.isna().sum()

ST004D01T     0
ST216Q03HA    0
IC001Q04TA    0
HISCED        0
ESCS          0
WEALTH        0
EMOSUPS       0
GFOFAIL       0
RESILIENCE    0
GCSELFEFF     0
BELONG        0
dtype: int64

In [9]:
# check the correlation among all variables
df.corr()

,ST004D01T,ST216Q03HA,IC001Q04TA,HISCED,ESCS,WEALTH,EMOSUPS,GFOFAIL,RESILIENCE,GCSELFEFF,BELONG
ST004D01T,1.000000,-0.056285,0.024200,0.021985,0.001477,0.025254,-0.079749,-0.153858,0.023170,0.007950,0.016550
ST216Q03HA,-0.056285,1.000000,0.045295,-0.059479,-0.098718,-0.079684,-0.137393,0.135016,-0.319046,-0.185577,-0.195096
IC001Q04TA,0.024200,0.045295,1.000000,-0.178039,-0.309483,-0.359884,-0.090546,-0.008740,-0.025828,-0.054400,-0.075262
HISCED,0.021985,-0.059479,-0.178039,1.000000,0.783221,0.334470,0.070089,0.000980,0.033242,0.112023,0.043826
ESCS,0.001477,-0.098718,-0.309483,0.783221,1.000000,0.667454,0.136971,0.024646,0.074975,0.179948,0.096773
WEALTH,0.025254,-0.079684,-0.359884,0.334470,0.667454,1.000000,0.115105,0.029425,0.059981,0.100221,0.114580
EMOSUPS,-0.079749,-0.137393,-0.090546,0.070089,0.136971,0.115105,1.000000,-0.025714,0.307471,0.159518,0.259570
GFOFAIL,-0.153858,0.135016,-0.008740,0.000980,0.024646,0.029425,-0.025714,1.000000,-0.125057,-0.001484,-0.178724
RESILIENCE,0.023170,-0.319046,-0.025828,0.033242,0.074975,0.059981,0.307471,-0.125057,1.000000,0.301858,0.313197
GCSELFEFF,0.007950,-0.185577,-0.054400,0.112023,0.179948,0.100221,0.159518,-0.001484,0.301858,1.000000,0.128204


In [10]:
# check for multilinearity among all variables 
from statsmodels.stats.outliers_influence import variance_inflation_factor
  
X = df[columns]
  
# VIF dataframe
vif_data = pd.DataFrame()
vif_data["feature"] = X.columns
  
# calculating VIF for each feature
vif_data["VIF"] = [variance_inflation_factor(X.values, i)
                          for i in range(len(X.columns))]
  
print(vif_data)

       feature        VIF
0      GFOFAIL   1.075630
1       WEALTH   2.283620
2   RESILIENCE   1.341328
3       BELONG   1.191673
4       HISCED  14.859314
5    ST004D01T   8.780753
6   IC001Q04TA   7.416169
7    GCSELFEFF   1.145078
8         ESCS   3.571231
9      EMOSUPS   1.166752
10  ST216Q03HA   6.302945


In [43]:
# the vif 'HISCED' is too high
# so this variable is not appropriate using in the regression model

df.drop(['HISCED'], axis=1)

,ST004D01T,ST216Q03HA,IC001Q04TA,ESCS,WEALTH,EMOSUPS,GFOFAIL,RESILIENCE,GCSELFEFF,BELONG
0,2,1.0,1,0.6747,-0.0996,0.2127,-0.6870,0.3693,-0.4609,-0.2133
1,2,2.0,2,-0.7566,-0.7221,-2.0851,-0.6133,-0.4684,0.7623,-0.0845
2,1,5.0,3,-2.5112,-3.6051,0.6364,-0.6133,-1.5248,-0.9477,-0.0824
3,2,2.0,1,-3.1843,-7.0376,1.0346,-0.6870,-0.0614,0.3313,-0.0358
4,2,2.0,1,-1.7557,-1.8375,-0.6576,0.1097,2.3693,0.6965,0.1704
...,...,...,...,...,...,...,...,...,...,...
601764,2,3.0,1,-1.8751,-0.6935,-0.6576,-1.8939,-0.0614,0.1904,-0.1182
601769,2,3.0,1,-3.0210,-2.4741,-0.6576,-1.8939,2.3693,0.1904,-0.9879
601774,2,3.0,2,-1.3621,-0.1630,-1.8624,-1.1597,-1.4033,-0.8238,-0.8735
601785,2,2.0,1,0.1740,-1.3599,1.0346,-0.9598,2.3693,0.1904,-0.6398


In [44]:
# in order to deal with the categorical variables
# we need to change the types to object

df['ST004D01T'] = df['ST004D01T'].astype(object)
df['ST216Q03HA'] = df['ST216Q03HA'].astype(object)
df['IC001Q04TA'] = df['IC001Q04TA'].astype(object)

In [45]:
dfDummies = pd.get_dummies(df[['ST004D01T', 'ST216Q03HA', 'IC001Q04TA']], drop_first=True)
df = pd.concat([df, dfDummies], axis=1)
df.drop(['ST004D01T', 'ST216Q03HA', 'IC001Q04TA'], axis=1)

,HISCED,ESCS,WEALTH,EMOSUPS,GFOFAIL,RESILIENCE,GCSELFEFF,BELONG,ST004D01T_2,ST216Q03HA_2.0,ST216Q03HA_3.0,ST216Q03HA_4.0,ST216Q03HA_5.0,IC001Q04TA_2,IC001Q04TA_3
0,6.0,0.6747,-0.0996,0.2127,-0.6870,0.3693,-0.4609,-0.2133,1,0,0,0,0,0,0
1,2.0,-0.7566,-0.7221,-2.0851,-0.6133,-0.4684,0.7623,-0.0845,1,1,0,0,0,1,0
2,2.0,-2.5112,-3.6051,0.6364,-0.6133,-1.5248,-0.9477,-0.0824,0,0,0,0,1,0,1
3,2.0,-3.1843,-7.0376,1.0346,-0.6870,-0.0614,0.3313,-0.0358,1,1,0,0,0,0,0
4,4.0,-1.7557,-1.8375,-0.6576,0.1097,2.3693,0.6965,0.1704,1,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
601764,2.0,-1.8751,-0.6935,-0.6576,-1.8939,-0.0614,0.1904,-0.1182,1,0,1,0,0,0,0
601769,4.0,-3.0210,-2.4741,-0.6576,-1.8939,2.3693,0.1904,-0.9879,1,0,1,0,0,0,0
601774,2.0,-1.3621,-0.1630,-1.8624,-1.1597,-1.4033,-0.8238,-0.8735,1,0,1,0,0,1,0
601785,6.0,0.1740,-1.3599,1.0346,-0.9598,2.3693,0.1904,-0.6398,1,1,0,0,0,0,0


In [14]:
# so that's all culumns we got
df.columns

Index(['ST004D01T', 'ST216Q03HA', 'IC001Q04TA', 'HISCED', 'ESCS', 'WEALTH',
       'EMOSUPS', 'GFOFAIL', 'RESILIENCE', 'GCSELFEFF', 'BELONG',
       'ST004D01T_2', 'ST216Q03HA_2.0', 'ST216Q03HA_3.0', 'ST216Q03HA_4.0',
       'ST216Q03HA_5.0', 'IC001Q04TA_2', 'IC001Q04TA_3'],
      dtype='object')

In [ ]:
# Do the standardized and save the file

In [18]:
df2 = df.copy()

In [23]:
# scaler = StandardScaler()
# scaler.fit(df)
# df2 = scaler.transform(df2)
# df22 = pd.DataFrame(df2)
# df22

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,-28.376886,-9.584227,-51.135995,-7.817539,1.445547,1.568110,0.343623,-0.847019,0.059229,-0.418104,-0.040948,1.643795,-26.432690,-27.065557,-54.080296,-225.266162,-244.287854,-132.323131
1,-28.376886,-8.790525,-2.234735,-8.434375,0.292542,1.043817,-1.933404,-0.770526,-0.773239,0.708388,0.083578,1.643795,14.871678,-27.065557,-54.080296,-225.266162,6374.097220,-132.323131
2,-60.410657,-6.409422,46.666525,-8.434375,-1.120901,-1.384355,0.763492,-0.770526,-1.823042,-0.866417,0.085608,-30.389976,-26.432690,-27.065557,-54.080296,5477.407653,-244.287854,1972.804234
3,-28.376886,-8.790525,-51.135995,-8.434375,-1.663127,-4.275337,1.158092,-0.847019,-0.368781,0.311464,0.130662,1.643795,14.871678,-27.065557,-54.080296,-225.266162,-244.287854,-132.323131
4,-28.376886,-8.790525,-51.135995,-8.125957,-0.512297,0.104385,-0.518809,-0.020122,2.046740,0.647790,0.330019,1.643795,14.871678,-27.065557,-54.080296,-225.266162,-244.287854,-132.323131
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
231853,-28.376886,-7.996824,-51.135995,-8.434375,-0.608482,1.067905,-0.518809,-2.099666,-0.368781,0.181703,0.050996,1.643795,-26.432690,24.902115,-54.080296,-225.266162,-244.287854,-132.323131
231854,-28.376886,-7.996824,-51.135995,-8.125957,-1.531578,-0.431784,-0.518809,-2.099666,2.046740,0.181703,-0.789844,1.643795,-26.432690,24.902115,-54.080296,-225.266162,-244.287854,-132.323131
231855,-28.376886,-7.996824,-2.234735,-8.434375,-0.195227,1.514712,-1.712717,-1.337637,-1.702301,-0.752313,-0.679241,1.643795,-26.432690,24.902115,-54.080296,-225.266162,6374.097220,-132.323131
231856,-28.376886,-8.790525,-51.135995,-7.817539,1.042201,0.506637,1.158092,-1.130160,2.046740,0.181703,-0.453295,1.643795,14.871678,-27.065557,-54.080296,-225.266162,-244.287854,-132.323131


In [29]:
# df22.columns = list(df)
# df22.to_csv("data_sd.csv")

# Linear Regression model

In [46]:
Y = df['GFOFAIL']
X = df.iloc[:, df.columns != 'GFOFAIL']

In [47]:
# standarlized the dataset
scaler = StandardScaler()
scaler.fit(X)
X = scaler.transform(X)

In [48]:
# split into training and testing datasets
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.20, random_state=53)

In [50]:
x_train

array([[-0.97966821, -0.36201606, -0.28108372, ..., -0.1832882 ,
        -0.17755359, -0.22766161],
       [ 1.02075375, -1.31685782, -0.28108372, ..., -0.1832882 ,
        -0.17755359, -0.22766161],
       [ 1.02075375, -1.31685782, -0.28108372, ..., -0.1832882 ,
        -0.17755359, -0.22766161],
       ...,
       [-0.97966821, -0.36201606, -0.28108372, ..., -0.1832882 ,
        -0.17755359, -0.22766161],
       [ 1.02075375,  0.5928257 , -0.28108372, ..., -0.1832882 ,
        -0.17755359, -0.22766161],
       [ 1.02075375, -1.31685782, -0.28108372, ..., -0.1832882 ,
        -0.17755359, -0.22766161]])

In [54]:
## construct linear regression model
clf = LinearRegression().fit(x_train,y_train)
 
## predicted values
y_pred = clf.predict(x_test)


In [56]:
print("coefficients =",clf.coef_)    # coefficients
print("intercept b =",clf.intercept_) # intercept
print('Mean Absolute Error =', mean_absolute_error(y_test, y_pred))
print('Mean Squared Error =', mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error =', np.sqrt(mean_squared_error(y_test, y_pred)))
print('R2 Score:', r2_score(y_test, y_pred))


coefficients = [ 5.41629275e+10  1.30561854e+10  1.94550246e+11 -4.02185953e-02
  5.02129343e-02  3.68968944e-02  1.96643235e-02 -6.74149775e-02
  4.28344714e-02 -1.56315957e-01 -5.41629275e+10 -5.92309251e+09
 -1.13143877e+10 -1.16204382e+10 -8.84284468e+09 -7.29029981e+10
 -1.83345441e+11]
intercept b = 0.0264501661927047
Mean Absolute Error = 0.7544366859612077
Mean Squared Error = 0.9095257571435947
Root Mean Squared Error = 0.9536905982254384
R2 Score: 0.07437957859945232


# Lasso Regression Model

In [76]:
lasso=Lasso(alpha=0.01,max_iter=10000)
lasso.fit(x_train,y_train)
print("Lasso training dataset score："+str(r2_score(y_train,lasso.predict(x_train))))
print("Lasso testing dataset score："+str(r2_score(y_test,lasso.predict(x_test))))

Lasso training dataset score：0.07125537285996952
Lasso testing dataset score：0.07264418294034136


算一下缺失值是否小于5%一下。otherwise，要看下是不是系统性缺失